# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 6 2022 10:43AM,241194,19,DEX0007124,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
1,Jul 6 2022 10:43AM,241194,19,DEX0007125,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
2,Jul 6 2022 10:43AM,241194,19,DEX0007126,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
3,Jul 6 2022 10:43AM,241194,19,DEX0007127,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
4,Jul 6 2022 10:43AM,241194,19,DEX0007128,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
5,Jul 6 2022 10:43AM,241194,19,DEX0007129,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
6,Jul 6 2022 10:43AM,241194,19,DEX0007130,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
7,Jul 6 2022 10:43AM,241194,19,DEX0007131,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
8,Jul 6 2022 10:43AM,241194,19,DEX0007132,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
9,Jul 6 2022 10:43AM,241194,19,DEX0007133,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
67,241190,Released,37
68,241191,Released,2
69,241192,Released,17
70,241193,Released,20
71,241194,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241190,NaN,NaN,37.0
241191,NaN,NaN,2.0
241192,NaN,NaN,17.0
241193,NaN,NaN,20.0
241194,NaN,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241080,0.0,0.0,42.0
241081,0.0,0.0,8.0
241082,0.0,0.0,63.0
241083,15.0,7.0,1.0
241087,0.0,15.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241080,0,0,42
241081,0,0,8
241082,0,0,63
241083,15,7,1
241087,0,15,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241080,0,0,42
1,241081,0,0,8
2,241082,0,0,63
3,241083,15,7,1
4,241087,0,15,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241080,,,42
1,241081,,,8
2,241082,,,63
3,241083,15,7,1
4,241087,,15,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 6 2022 10:43AM,241194,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
12,Jul 6 2022 10:42AM,241193,15,"Brookfield Pharmaceuticals, LLC"
32,Jul 6 2022 10:40AM,241192,10,Eywa Pharma Inc.
49,Jul 6 2022 10:38AM,241190,15,"Alliance Pharma, Inc."
86,Jul 6 2022 10:30AM,241191,10,ISDIN Corporation
88,Jul 6 2022 10:30AM,241189,10,ISDIN Corporation
134,Jul 6 2022 10:26AM,241188,10,ISDIN Corporation
203,Jul 6 2022 10:24AM,241187,10,ISDIN Corporation
260,Jul 6 2022 10:18AM,241186,10,ISDIN Corporation
269,Jul 6 2022 10:14AM,241184,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 6 2022 10:43AM,241194,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,12
1,Jul 6 2022 10:42AM,241193,15,"Brookfield Pharmaceuticals, LLC",,,20
2,Jul 6 2022 10:40AM,241192,10,Eywa Pharma Inc.,,,17
3,Jul 6 2022 10:38AM,241190,15,"Alliance Pharma, Inc.",,,37
4,Jul 6 2022 10:30AM,241191,10,ISDIN Corporation,,,2
5,Jul 6 2022 10:30AM,241189,10,ISDIN Corporation,,,46
6,Jul 6 2022 10:26AM,241188,10,ISDIN Corporation,,,69
7,Jul 6 2022 10:24AM,241187,10,ISDIN Corporation,,,57
8,Jul 6 2022 10:18AM,241186,10,ISDIN Corporation,,,9
9,Jul 6 2022 10:14AM,241184,10,ISDIN Corporation,,,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 10:43AM,241194,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,12,,
1,Jul 6 2022 10:42AM,241193,15,"Brookfield Pharmaceuticals, LLC",20,,
2,Jul 6 2022 10:40AM,241192,10,Eywa Pharma Inc.,17,,
3,Jul 6 2022 10:38AM,241190,15,"Alliance Pharma, Inc.",37,,
4,Jul 6 2022 10:30AM,241191,10,ISDIN Corporation,2,,
5,Jul 6 2022 10:30AM,241189,10,ISDIN Corporation,46,,
6,Jul 6 2022 10:26AM,241188,10,ISDIN Corporation,69,,
7,Jul 6 2022 10:24AM,241187,10,ISDIN Corporation,57,,
8,Jul 6 2022 10:18AM,241186,10,ISDIN Corporation,9,,
9,Jul 6 2022 10:14AM,241184,10,ISDIN Corporation,14,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 10:43AM,241194,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,12,,
1,Jul 6 2022 10:42AM,241193,15,"Brookfield Pharmaceuticals, LLC",20,,
2,Jul 6 2022 10:40AM,241192,10,Eywa Pharma Inc.,17,,
3,Jul 6 2022 10:38AM,241190,15,"Alliance Pharma, Inc.",37,,
4,Jul 6 2022 10:30AM,241191,10,ISDIN Corporation,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 10:43AM,241194,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,12.0,NaN,NaN
1,Jul 6 2022 10:42AM,241193,15,"Brookfield Pharmaceuticals, LLC",20.0,NaN,NaN
2,Jul 6 2022 10:40AM,241192,10,Eywa Pharma Inc.,17.0,NaN,NaN
3,Jul 6 2022 10:38AM,241190,15,"Alliance Pharma, Inc.",37.0,NaN,NaN
4,Jul 6 2022 10:30AM,241191,10,ISDIN Corporation,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 10:43AM,241194,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,12.0,0.0,0.0
1,Jul 6 2022 10:42AM,241193,15,"Brookfield Pharmaceuticals, LLC",20.0,0.0,0.0
2,Jul 6 2022 10:40AM,241192,10,Eywa Pharma Inc.,17.0,0.0,0.0
3,Jul 6 2022 10:38AM,241190,15,"Alliance Pharma, Inc.",37.0,0.0,0.0
4,Jul 6 2022 10:30AM,241191,10,ISDIN Corporation,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,8199136,563.0,2.0,3.0
12,1205666,28.0,2.0,0.0
15,482383,57.0,0.0,0.0
16,1687865,11.0,5.0,0.0
18,241165,1.0,0.0,0.0
19,964583,69.0,42.0,5.0
20,1205560,96.0,32.0,15.0
21,241183,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,8199136,563.0,2.0,3.0
1,12,1205666,28.0,2.0,0.0
2,15,482383,57.0,0.0,0.0
3,16,1687865,11.0,5.0,0.0
4,18,241165,1.0,0.0,0.0
5,19,964583,69.0,42.0,5.0
6,20,1205560,96.0,32.0,15.0
7,21,241183,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,563.0,2.0,3.0
1,12,28.0,2.0,0.0
2,15,57.0,0.0,0.0
3,16,11.0,5.0,0.0
4,18,1.0,0.0,0.0
5,19,69.0,42.0,5.0
6,20,96.0,32.0,15.0
7,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,563.0
1,12,Released,28.0
2,15,Released,57.0
3,16,Released,11.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,5.0,15.0,0.0
Executing,2.0,2.0,0.0,5.0,0.0,42.0,32.0,0.0
Released,563.0,28.0,57.0,11.0,1.0,69.0,96.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,5.0,15.0,0.0
1,Executing,2.0,2.0,0.0,5.0,0.0,42.0,32.0,0.0
2,Released,563.0,28.0,57.0,11.0,1.0,69.0,96.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,5.0,15.0,0.0
1,Executing,2.0,2.0,0.0,5.0,0.0,42.0,32.0,0.0
2,Released,563.0,28.0,57.0,11.0,1.0,69.0,96.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()